In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

In [12]:
!pip install pulp
!pip install json

   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.8/16.4 MB 2.4 MB/s eta 0:00:07
   --- ------------------------------------ 1.3/16.4 MB 2.5 MB/s eta 0:00:07
   ----- ---------------------------------- 2.1/16.4 MB 2.9 MB/s eta 0:00:05
   ------ --------------------------------- 2.6/16.4 MB 2.9 MB/s eta 0:00:05
   ------- -------------------------------- 3.1/16.4 MB 2.8 MB/s eta 0:00:05
   --------- ------------------------------ 3.9/16.4 MB 3.0 MB/s eta 0:00:05
   ----------- ---------------------------- 4.7/16.4 MB 2.9 MB/s eta 0:00:04
   ------------- -------------------------- 5.5/16.4 MB 3.0 MB/s eta 0:00:04
   ---------------- ----------------------- 6.6/16.4 MB 3.3 MB/s eta 0:00:04
   ----------------- ---------------------- 7.3/16.4 MB 3.3 MB/s eta 0:00:03
   -------------------- ------------------- 8.4/16.4 MB 3.5 MB/s eta 0:00:03
   ----------

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [13]:
import pandas as pd
import pulp
import json


In [15]:
flows = pd.read_csv("flows_peak.csv")
signals = pd.read_csv("signals_current.csv")
with open("constraints.json") as f:
    constraints = json.load(f)


In [16]:
model = pulp.LpProblem("TrafficSignalOptimization", pulp.LpMinimize)

intersections = flows['intersection_id'].unique()

# Переменные: длина цикла и фазы
cycle = {i: pulp.LpVariable(f"cycle_{i}", constraints['min_cycle_sec'], constraints['max_cycle_sec'], cat='Continuous')
         for i in intersections}
green_main = {i: pulp.LpVariable(f"green_main_{i}", constraints['min_green_sec'], cycle[i], cat='Continuous')
              for i in intersections}
green_sec = {i: pulp.LpVariable(f"green_sec_{i}", constraints['min_green_sec'], cycle[i], cat='Continuous')
             for i in intersections}

# Оффсет начала цикла (координация)
offset = {i: pulp.LpVariable(f"offset_{i}", 0, cycle[i], cat='Continuous')
          for i in intersections}


In [17]:
delay_terms = []
for _, row in flows.iterrows():
    i = row['intersection_id']
    flow = row['intensity_veh_per_hr']
    bus_share = row['bus_share']
    
    # Приоритет автобусов: добавка к зелёному (перекрёсток 2, направление N-S)
    bus_bonus = constraints['min_extra_green_sec'] if i == 2 and constraints.get("priority_direction") == "NS" else 0

    # Простая модель: чем меньше зелёный — тем больше задержка
    green_time = green_main[i] if row['approach'] in ['N', 'S'] else green_sec[i]
    delay = (cycle[i] - green_time - bus_bonus) * flow

    delay_terms.append(delay)

model += pulp.lpSum(delay_terms)


In [18]:
# Пешеходы
for i in intersections:
    model += green_main[i] >= constraints['pedestrian_green_sec']
    model += green_sec[i] >= constraints['pedestrian_green_sec']

# Оффсет: синхронизация зелёной волны вдоль главной оси
# Предположим расстояние 300 м между узлами, скорость 36 км/ч => 30 сек проезд
travel_time = 30  # секунд между перекрёстками

model += offset[2] == offset[1] + travel_time
model += offset[3] == offset[2] + travel_time

# Координация по главной оси (N-S)
# Зелёный стартует на всех примерно в одно и то же "движущееся окно"
for i in [1, 2, 3]:
    model += offset[i] + green_main[i] <= cycle[i]


In [19]:
solver = pulp.PULP_CBC_CMD(msg=1)
model.solve(solver)

for i in intersections:
    print(f"Перекрёсток {i}: цикл = {cycle[i].value():.1f} сек, главная фаза = {green_main[i].value():.1f}, второстепенная = {green_sec[i].value():.1f}, offset = {offset[i].value():.1f}")


Перекрёсток 1: цикл = 60.0 сек, главная фаза = 15.0, второстепенная = 15.0, offset = 0.0
Перекрёсток 2: цикл = 60.0 сек, главная фаза = 15.0, второстепенная = 15.0, offset = 0.0
Перекрёсток 3: цикл = 60.0 сек, главная фаза = 15.0, второстепенная = 15.0, offset = 0.0
